In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# Assume we have some sample data to work with
# Note: This is toy data for demonstration; in a real case, these would be extracted from actual data

# Sample data inputs
num_videos = 1000  # Total unique videos
num_languages = 5  # Total unique languages
num_devices = 3    # Total unique devices
num_days = 7       # Days of the week
embedding_dim = 10 # Embedding dimension for IDs

# Sample Video Data (for one user-video pair)
video_id_input = 123         # Index of the video ID
duration_input = 15 / 120    # Normalized duration (0 to 1)
language_input = 1           # Language index (e.g., 1 for English)
title_embedding = np.random.rand(768)  # Pre-trained title embedding (BERT, for example)
tags_embedding = np.random.rand(100)   # Pre-trained tags embedding (CBOW)

# Sample User Data
age_input = 25 / 100          # Normalized age
gender_input = 1              # Gender index (e.g., 1 for Female)
location_input = 2            # Location index (e.g., 2 for US)
time_of_day_input = [np.sin(20), np.cos(20)]  # Cyclic encoding for time (8 PM = 20th hour)
device_input = 2              # Device index (e.g., 2 for Mobile)
day_of_week_input = 6         # Day index (e.g., 6 for Saturday)

# Sample Historical Interactions
search_history_embedding = np.random.rand(768)  # Averaged BERT embeddings of search history
liked_video_embedding = np.random.rand(10)      # Averaged embeddings of liked videos
watched_video_embedding = np.random.rand(10)    # Averaged embeddings of watched videos

# 1. Define Input Layers for each feature
video_id = Input(shape=(1,), dtype='int32', name='video_id')
duration = Input(shape=(1,), name='duration')
language = Input(shape=(1,), dtype='int32', name='language')
age = Input(shape=(1,), name='age')
gender = Input(shape=(1,), dtype='int32', name='gender')
location = Input(shape=(1,), dtype='int32', name='location')
time_of_day = Input(shape=(2,), name='time_of_day')  # Sinusoidal encoding input for time of day
device = Input(shape=(1,), dtype='int32', name='device')
day_of_week = Input(shape=(1,), dtype='int32', name='day_of_week')

# Historical Interactions and Textual Embeddings
title = Input(shape=(768,), name='title')
tags = Input(shape=(100,), name='tags')
search_history = Input(shape=(768,), name='search_history')
liked_videos = Input(shape=(10,), name='liked_videos')
watched_videos = Input(shape=(10,), name='watched_videos')

# 2. Embedding Layers
video_id_embedding = Embedding(input_dim=num_videos, output_dim=embedding_dim)(video_id)
language_embedding = Embedding(input_dim=num_languages, output_dim=5)(language)
gender_embedding = Embedding(input_dim=2, output_dim=2)(gender)
location_embedding = Embedding(input_dim=10, output_dim=5)(location)
device_embedding = Embedding(input_dim=num_devices, output_dim=3)(device)
day_of_week_embedding = Embedding(input_dim=num_days, output_dim=3)(day_of_week)


# Flatten all embeddings to make them 1-dimensional
video_id_embedding = Flatten()(video_id_embedding)
language_embedding = Flatten()(language_embedding)
gender_embedding = Flatten()(gender_embedding)
location_embedding = Flatten()(location_embedding)
device_embedding = Flatten()(device_embedding)
day_of_week_embedding = Flatten()(day_of_week_embedding)

# 3. Concatenate All Features into a Single Vector
all_features = Concatenate()([
    video_id_embedding,
    duration,
    language_embedding,
    title,
    tags,
    age,
    gender_embedding,
    location_embedding,
    time_of_day,
    device_embedding,
    day_of_week_embedding,
    search_history,
    liked_videos,
    watched_videos
])

# 4. Fully Connected Layers (Dense Layers)
x = Dense(512, activation='relu')(all_features)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)

# Output Layer - Predicts the relevance score between 0 and 1
output = Dense(1, activation='sigmoid', name='output')(x)

# 5. Define and Compile Model
model = Model(
    inputs=[
        video_id, duration, language, title, tags,
        age, gender, location, time_of_day, device,
        day_of_week, search_history, liked_videos, watched_videos
    ],
    outputs=output
)

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

# 6. Example Input Data to Model
input_data = {
    'video_id': np.array([video_id_input]),
    'duration': np.array([duration_input]),
    'language': np.array([language_input]),
    'title': np.array([title_embedding]),
    'tags': np.array([tags_embedding]),
    'age': np.array([age_input]),
    'gender': np.array([gender_input]),
    'location': np.array([location_input]),
    'time_of_day': np.array([time_of_day_input]),
    'device': np.array([device_input]),
    'day_of_week': np.array([day_of_week_input]),
    'search_history': np.array([search_history_embedding]),
    'liked_videos': np.array([liked_video_embedding]),
    'watched_videos': np.array([watched_video_embedding])
}

# 7. Predict relevance score for this sample data
prediction = model.predict(input_data)
print("Predicted Relevance Score:", prediction[0][0])